In [1]:
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

from pyarrow.dataset import HivePartitioning

from model.OutpatientsModel import OutpatientsModel

In [2]:
dataset = "data/synthetic"
model_run = f"test/20220110_104353"
m = OutpatientsModel(f"{dataset}/results/{model_run}")
op = pq.read_pandas(f"{dataset}/op.parquet").to_pandas()
op.drop(["hsagrp"], axis = "columns", inplace = True)

In [3]:
[sum(op.attendances), sum(op.tele_attendances), len(op.index)]

[548059, 17077, 562543]

In [4]:
selected_variant, results = m.run(0)
results

,index,attendances,tele_attendances
0,0,2,0
1,1,1,0
2,2,1,0
3,3,1,0
4,5,2,0
...,...,...,...
385521,562535,2,0
385522,562536,1,1
385523,562537,1,0
385524,562538,3,0


The "attendances" column shows how many outpatient attendances there were for each row of data.

In [5]:
row_count = pd.merge(
  op.value_counts("attendances").to_frame("baseline"),
  results.value_counts("attendances").to_frame("results"),
  left_index = True,
  right_index = True,
  how = "outer"
).fillna(0)

row_count.loc["total"] = [
  np.sum(row_count.baseline * row_count.index),
  np.sum(row_count.results * row_count.index)
]

row_count.astype(int)

,baseline,results
attendances,,
0,17072,18188
1,542930,196819
2,2494,111031
3,47,42743
4,0,12739
5,0,3183
6,0,657
7,0,131
8,0,25
